# PIPELINE TESTING

In [1]:
#
import os
import numpy as np
import pandas as pd
from pandas import Timestamp
import seaborn as sns
import matplotlib.pyplot as plt

#
from zipline.api import order_target_percent, record, symbol, set_benchmark, get_open_orders
import zipline
import datetime
import pytz
from zipline.data.bundles import load
from trading_calendars import get_calendar
import pyfolio as pf
#
%load_ext zipline

# Check Symbols of Bundle

In [14]:
trading_calendar=get_calendar('24/7')

now = Timestamp.utcnow()
bundle = load('NFTU', os.environ, now)
symbols = set(str(asset.symbol) 
              for asset in bundle.asset_finder.retrieve_all(
                           bundle.asset_finder.equities_sids))

print(symbols)

{'MANA', 'MATIC', 'ADA', 'SOL', 'TFUEL', 'FLOW', 'AVAX'}


# Simple Strategy

In [9]:
selected_coin = 'ADA'
shortest_period = 7
longest_period = 14
holding_period = 2

def initialize(context):
    context.i = 0
    context.holding_days = 0
    context.bought = False
    set_benchmark(symbol(selected_coin))
    context.asset = symbol(selected_coin)

def handle_data(context, data):
    # skip days for moving average
    context.i += 1
    if context.i < longest_period:
        return
    if context.bought:
        context.holding_days += 1
        
    # trading algorithm
    short_ma = data.history(context.asset, 'close', bar_count=shortest_period, frequency='1d').mean()
    long_ma = data.history(context.asset, 'close', bar_count=longest_period, frequency='1d').mean()

    open_orders = get_open_orders()
    
    # buy signal
    if context.asset not in open_orders:
        if short_ma > long_ma and context.holding_days == 0 and not context.bought:
            order_target_percent(context.asset, 1.0)
            context.bought = True
        
    # sell signal
    if (short_ma < long_ma or context.holding_days == holding_period) and context.bought:
        order_target_percent(context.asset, 0.0)
        context.holding_days = 0
        context.bought = False

    # record price
    record(close=data.current(symbol(selected_coin), 'close'),
          short_ma=short_ma,
          long_ma=long_ma)

In [12]:
# date range MUST BE pd.Timestamp
start_date = pd.Timestamp('2017-11-09', tz='utc')
end_date = pd.Timestamp('2022-04-11', tz='utc')

# performance of algorithm once run
perf = zipline.run_algorithm(start=start_date,
                            end=end_date,
                            trading_calendar=get_calendar('24/7'), 
                            initialize=initialize,
                            capital_base=1000,
                            handle_data=handle_data,
                            bundle='NFTU',
                            data_frequency='daily')
perf.head()
perf.tail()

AssertionError: All readers must share target trading_calendar. Reader=<zipline.data.minute_bars.BcolzMinuteBarReader object at 0x7fd6a548bd60> for type=<class 'zipline.assets._assets.Equity'> uses calendar=<exchange_calendars.always_open.AlwaysOpenCalendar object at 0x7fd6a5738f70> which does not match the desired shared calendar=<trading_calendars.always_open.AlwaysOpenCalendar object at 0x7fd6a5662cd0> 

In [13]:
# Extract algo returns and benchmark returns
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
benchmark_period_return = perf['benchmark_period_return']

# Convert benchmark returns to daily returns
#daily_returns = (1 + benchmark_period_return) / (1 + benchmark_period_return.shift()) - 1
daily_benchmark_returns = np.exp(np.log(benchmark_period_return + 1.0).diff()) - 1

# Create tear sheet
pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, benchmark_rets=daily_benchmark_returns, live_start_date='2021-01-27')

NameError: name 'perf' is not defined